In [1]:
import pandas as pd
import numpy as np
from evoml.subspacing import FeatureStackerFEGT, FeatureStackerFEMPO, FeatureStackerFECV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
#data = pd.read_csv('datasets/GAMETES.csv',sep='\t')
data = pd.read_csv('datasets/GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_EDM-1_1.txt',sep='\t')
headers_ = list(data.columns)
features = data[headers_[0:-1]]
output = data[headers_[-1]]

In [ ]:
from sklearn.tree import DecisionTreeClassifier


## Expriment 1 
----

In [5]:
X_train.shape

(1200, 1000)

In [50]:
def check_for_benchmarks():
    n_estimators=10
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

    # The exploration of the dataset by benchmark algorithms
    clf = DecisionTreeClassifier(random_state=34092)
    clf.fit(X_train, y_train)
    pred_DTC = clf.predict(X_test)
    a = clf.score(X_test, y_test)
    print('Base DecisionTreeClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

    clf = RandomForestClassifier(random_state=34092, n_estimators=n_estimators)
    clf.fit(X_train, y_train)
    pred_RFC = clf.predict(X_test)
    b = clf.score(X_test, y_test)
    print('Base RandomForestClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

    clf = GradientBoostingClassifier(random_state=34092, n_estimators=n_estimators)
    clf.fit(X_train, y_train)
    pred_GBC = clf.predict(X_test)
    c = clf.score(X_test, y_test)
    print('Base GradientBoostingClassifier accuracy: {}'.format(clf.score(X_test, y_test)))

    print('')
    return a,b,c

In [51]:
FeatureStackerFECV?

In [ ]:
all_acc = []
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
max_features = int(pd.np.sqrt(X_train.shape[1]))
for i in range(0,10):
    print(i)
    X_train, X_test, y_train, y_test = train_test_split(features, output, stratify=output,
                                                            train_size=0.75, test_size=0.25)
    a, b, c = check_for_benchmarks()
    clf_dt = DecisionTreeClassifier(max_depth=None)
#     clf_lr = LogisticRegressionCV()
    #clf = FeatureStackerFEMPO(base_estimator=clf_dt, model_type = 1, N_individual=5, ngen=10, verbose_flag = True, N_population=10, maxOrmin = 1)
    clf = FeatureStackerFECV(base_estimator=clf_dt, model_type = 'classification', N_individual=10,
                             ngen=10, verbose_flag = True, N_population=50, maxOrMin = 1
                            ,featMax = 33, featMin=1, folds_CV=5)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    d = accuracy_score(pred,y_test)
    all_acc.append([a,b,c,d])

0
Base DecisionTreeClassifier accuracy: 0.4775
Base RandomForestClassifier accuracy: 0.4975
Base GradientBoostingClassifier accuracy: 0.4625

gen	nevals	avg     	min     	max     
0  	50    	0.495783	0.471667	0.529167
1  	32    	0.504967	0.476667	0.529167
2  	33    	0.51315 	0.49    	0.54    
3  	30    	0.516   	0.490833	0.54    
4  	35    	0.521633	0.495   	0.54    
5  	32    	0.527967	0.509167	0.548333
6  	34    	0.52925 	0.5125  	0.551667
7  	36    	0.530967	0.5075  	0.555833
8  	35    	0.537667	0.503333	0.558333
9  	32    	0.542633	0.518333	0.560833
10 	35    	0.54935 	0.531667	0.565833
1
Base DecisionTreeClassifier accuracy: 0.5325
Base RandomForestClassifier accuracy: 0.5175
Base GradientBoostingClassifier accuracy: 0.49



In [126]:
clf.hof[0].fitness

deap.creator.FitnessMax((0.5558333333333334,))

In [127]:
from sklearn.metrics import accuracy_score
pred = clf.predict(X_test)
accuracy_score(pred, y_test)

0.51000000000000001

In [128]:
shapes = [eg.X.shape[1] for eg in clf.hof[0]]

In [138]:
pd.DataFrame(all_acc).median()

0    0.50500
1    0.48250
2    0.49875
3    0.48875
dtype: float64

In [130]:
from evoml.subspacing.util import EstimatorGene
hof = []
bagged_hof = []
for i,eg in enumerate(clf.hof[0]):
    xcols = eg.X.columns
    X = X_train.sample(frac=1, replace=True)
    X = X.loc[:,xcols]
    eg_ = EstimatorGene(X,y_train,[],[],clf_dt)
    bagged_hof.append(eg_)
hof.append(bagged_hof)

In [131]:
clf.hof = hof[:]

In [132]:
pred = clf.predict(X_test)
accuracy_score(pred, y_test)

0.47749999999999998